<script async src="https://www.googletagmanager.com/gtag/js?id=UA-59152712-8"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());

  gtag('config', 'UA-59152712-8');
</script>

# `GiRaFFE_NRPy_staggered`: Solving the Induction Equation

## Author: Patrick Nelson

**Notebook Status:** <font color='green'><b>Validated</b></font>

**Validation Notes:** This code is a port from the old `GiRaFFE`

### NRPy+ Source Code for this module:
* [GiRaFFE_NRPy/GiRaFFE_NRPy_staggered_Afield_flux.py](../../edit/in_progress/GiRaFFE_NRPy/GiRaFFE_NRPy_staggered_Afield_flux.py)

## Introduction

This notebook presents an alternate algorithm for computing the flux term in the GRFFE induction equation in a staggered prescription. It is a direct port of the old `GiRaFFE` implementation, which implements the algorithm described in [Del Zanna, et al. (2003)](https://arxiv.org/abs/astro-ph/0210618)

<a id='toc'></a>

# Table of Contents
$$\label{toc}$$

This notebook is organized as follows

1. [Step 1](#prelim): Preliminaries
1. [Step 2](#code): Write the C code
    1. [Step 2.a](#declaration): Write the function declaration and map inputs to $A_z$ formula
    1. [Step 2.a](#declaration): Write the function declaration
1. [Step 3](#code_validation): Code Validation
1. [Step 4](#latex_pdf_output): Output this notebook to $\LaTeX$-formatted PDF file

<a id='prelim'></a>

# Step 1: Preliminaries \[Back to [top](#toc)\]
$$\label{prelim}$$

This first block of code just sets up a subdirectory within `GiRaFFE_standalone_Ccodes/` to which we will write the C code.

In [1]:
# Step 0: Add NRPy's directory to the path
# https://stackoverflow.com/questions/16780014/import-file-from-parent-directory
import os,sys
nrpy_dir_path = os.path.join("..")
if nrpy_dir_path not in sys.path:
    sys.path.append(nrpy_dir_path)

import cmdline_helper as cmd     # NRPy+: Multi-platform Python command-line interface
Ccodesdir = "GiRaFFE_standalone_Ccodes/RHSs"
cmd.mkdir(os.path.join(Ccodesdir))

<a id='code'></a>

# Step 2: Write the C code \[Back to [top](#toc)\]
$$\label{code}$$

Our goal here is to compute the part of A_i_rhs that excludes the gauge terms:
$$
\partial_t A_z = \psi^{6} (v^z B^x - v^x B^z)
$$
As can be seen above, we can do this by coding just one component ($z$) and permuting the inputs.

<a id='declaration'></a>

## Step 2.a: Write the function declaration and map inputs to $A_z$ formula \[Back to [top](#toc)\]
$$\label{declaration}$$

Our inputs include the component of $A_i$ (as a number between 1 and 3), the usual NRPy+ `paramstruct`, the reconstructed Valencia three-velocity on the left and right faces (as contained in the `gf_and_gz_struct`s), the square root of the metric determinant $\sqrt{\gamma} = \psi^6$, and the characteristic speeds. The output is the right-hand side (RHS) of the $A_i$ equation.

We then use the `#include` to set parameters; if the declaration of a certain variable is otherwise not seen in this file, it was performed here.

Next, we will define the *actual* directions that correspond to `v1` and `v2` from the input `A_dirn`.
* If `A_dirn=1`, then `v1_offset=1` (v1=VY) and `v2_offset=2` (v2=VZ)
* If `A_dirn=2`, then `v1_offset=2` (v1=VZ) and `v2_offset=0` (v2=VX)
* If `A_dirn=3`, then `v1_offset=0` (v1=VX) and `v2_offset=1` (v2=VY)

In [2]:
%%writefile $Ccodesdir/A_i_rhs_no_gauge_terms.h
/* Compute the part of A_i_rhs that excludes the gauge terms. I.e., we set
 *   A_i_rhs = \partial_t A_i = \psi^{6} (v^z B^x - v^x B^z)   here.
 */
static void A_i_rhs_no_gauge_terms(const int A_dirn,const paramstruct *params,gf_and_gz_struct *out_prims_r,gf_and_gz_struct *out_prims_l,
                                   REAL *psi6_pointer,REAL *cmax_1,REAL *cmin_1,REAL *cmax_2,REAL *cmin_2, REAL *A3_rhs) {
  #include "../set_Cparameters.h"

  // If A_dirn=1, then v1_offset=1 (v1=VY) and v2_offset=2 (v2=VZ)
  // If A_dirn=2, then v1_offset=2 (v1=VZ) and v2_offset=0 (v2=VX)
  // If A_dirn=3, then v1_offset=0 (v1=VX) and v2_offset=1 (v2=VY)
  const int v1_offset  = ((A_dirn-1)+1)%3,        v2_offset = ((A_dirn-1)+2)%3;


Overwriting GiRaFFE_standalone_Ccodes/RHSs/A_i_rhs_no_gauge_terms.h


With the directions defined, we now have a mapping for the formulae for each of the $A_i$ RHSs onto the $A_z$ RHS formula. We use that here to set the pointers to the reconstructed velocities and magnetic fields that we have already computed, allowing this function to be completely general while using a single naming scheme for convenience.

In [3]:
%%writefile -a $Ccodesdir/A_i_rhs_no_gauge_terms.h

  const REAL *v1rr=out_prims_r[VXR+v1_offset].gf, *v2rr=out_prims_r[VXR+v2_offset].gf;
  const REAL *v1rl=out_prims_l[VXR+v1_offset].gf, *v2rl=out_prims_l[VXR+v2_offset].gf;
  const REAL *v1lr=out_prims_r[VXL+v1_offset].gf, *v2lr=out_prims_r[VXL+v2_offset].gf;
  const REAL *v1ll=out_prims_l[VXL+v1_offset].gf, *v2ll=out_prims_l[VXL+v2_offset].gf;

  const REAL *B1r=out_prims_r[BX_STAGGER+v1_offset].gf, *B1l=out_prims_l[BX_STAGGER+v1_offset].gf;
  const REAL *B2r=out_prims_r[BX_STAGGER+v2_offset].gf, *B2l=out_prims_l[BX_STAGGER+v2_offset].gf;


Appending to GiRaFFE_standalone_Ccodes/RHSs/A_i_rhs_no_gauge_terms.h


There is one more wrinkle to consider in mapping $A_i$ to $A_z$, which is where the reconstructions of the velocities and magnetic fields are stored. So, we up two arrays of offsets to coordinate this. 
* $\bar{v}^i$ dependencies:
    * In the case of `Ax_rhs`, we need `v{y,z}{r,l}` at $(i,j+1/2,k+1/2)$.
        *    However, `v{y,z}{r,l}{r,l}` are defined at $(i,j-1/2,k-1/2)$, so
        *    `v{y,z}{r,l}` at $(i,j+1/2,k+1/2)$ is stored at `v{y,z}{r,l}{r,l}(i,j+1,k+1)`.
    * In the case of `Ay_rhs`, we need `v{x,z}{r,l}` at $(i+1/2,j,k+1/2)$.
        *    However, `v{x,z}{r,l}{r,l}` are defined at $(i-1/2,j,k-1/2)$, so
        *    `v{x,z}{r,l}` at $(i+1/2,j,k+1/2)$ is stored at `v{x,z}{r,l}{r,l}(i+1,j,k+1)`.
    * In the case of `Az_rhs`, we need `v{x,y}{r,l}` at $(i+1/2,j+1/2,k)$.
        *    However, `v{x,y}{r,l}{r,l}` are defined at $(i-1/2,j-1/2,k)$, so
        *    `v{x,y}{r,l}` at $(i+1/2,j+1/2,k)$ is stored at `v{x,y}{r,l}{r,l}(i+1,j+1,k)`.
* $B^i$ dependencies:
    * In the case of `Ax_rhs`, we need `B{y,z}_stagger{r,l}` at $(i,j+1/2,k+1/2)$.
        *    However, `By_stagger{r,l}` is defined at $(i,j+1/2,k-1/2)$, and
        *             `Bz_stagger{r,l}` is defined at $(i,j-1/2,k+1/2)$, so
        *             `By_stagger{r,l}` at $(i,j+1/2,k+1/2)$ is stored at `By_stagger{r,l}(i,j,k+1)`, and
        *             `Bz_stagger{r,l}` at $(i,j+1/2,k+1/2)$ is stored at `Bz_stagger{r,l}(i,j+1,k)`.
    * In the case of `Ay_rhs`, we need `B{z,x}_stagger{r,l}` at $(i+1/2,j,k+1/2)$.
        *    However, `Bz_stagger{r,l}` is defined at $(i-1/2,j,k+1/2)$, and
        *             `Bx_stagger{r,l}` is defined at $(i+1/2,j,k-1/2)$, so
        *             `Bz_stagger{r,l}` at $(i+1/2,j,k+1/2)$ is stored at `Bz_stagger{r,l}(i+1,j,k)`, and
        *             `Bx_stagger{r,l}` at $(i+1/2,j,k+1/2)$ is stored at `Bx_stagger{r,l}(i,j,k+1)`.
    * In the case of `Az_rhs`, we need `B{x,y}{r,l}_stagger` at $(i+1/2,j+1/2,k)$.
        *    However, `Bx_stagger{r,l}` is defined at $(i+1/2,j-1/2,k)$, and
        *             `By_stagger{r,l}` is defined at $(i-1/2,j+1/2,k)$, so
        *             `Bx_stagger{r,l}` at $(i+1/2,j+1/2,k)$ is stored at `Bx_stagger{r,l}(i,j+1,k)`, and
        *             `By_stagger{r,l}` at $(i+1/2,j+1/2,k)$ is stored at `By_stagger{r,l}(i+1,j,k)`.


In [4]:
%%writefile -a $Ccodesdir/A_i_rhs_no_gauge_terms.h

  /**** V DEPENDENCIES ****/
  /* In the case of Ax_rhs, we need v{y,z}{r,l} at (i,j+1/2,k+1/2).
   *    However, v{y,z}{r,l}{r,l} are defined at (i,j-1/2,k-1/2), so
   *    v{y,z}{r,l} at (i,j+1/2,k+1/2) is stored at v{y,z}{r,l}{r,l}(i,j+1,k+1).
   * In the case of Ay_rhs, we need v{x,z}{r,l} at (i+1/2,j,k+1/2).
   *    However, v{x,z}{r,l}{r,l} are defined at (i-1/2,j,k-1/2), so
   *    v{x,z}{r,l} at (i+1/2,j,k+1/2) is stored at v{x,z}{r,l}{r,l}(i+1,j,k+1).
   * In the case of Az_rhs, we need v{x,y}{r,l} at (i+1/2,j+1/2,k).
   *    However, v{x,y}{r,l}{r,l} are defined at (i-1/2,j-1/2,k), so
   *    v{x,y}{r,l} at (i+1/2,j+1/2,k) is stored at v{x,y}{r,l}{r,l}(i+1,j+1,k). */
  static const int vs_ijk_offset[4][3] = { {0,0,0} , {0,1,1} , {1,0,1} , {1,1,0} }; // Note that vs_ijk_offset[0] is UNUSED; we choose a 1-offset for convenience.

  /**** B DEPENDENCIES ****/
  /* In the case of Ax_rhs, we need B{y,z}{r,l} at (i,j+1/2,k+1/2).
   *    However, By_stagger{r,l} is defined at (i,j+1/2,k-1/2), and
   *             Bz_stagger{r,l} is defined at (i,j-1/2,k+1/2), so
   *             By_stagger{r,l} at (i,j+1/2,k+1/2) is stored at By_stagger{r,l}(i,j,k+1), and
   *             Bz_stagger{r,l} at (i,j+1/2,k+1/2) is stored at Bz_stagger{r,l}(i,j+1,k).
   * In the case of Ay_rhs, we need B{z,x}_stagger{r,l} at (i+1/2,j,k+1/2).
   *    However, Bz_stagger{r,l} is defined at (i-1/2,j,k+1/2), and
   *             Bx_stagger{r,l} is defined at (i+1/2,j,k-1/2), so
   *             Bz_stagger{r,l} at (i+1/2,j,k+1/2) is stored at Bz_stagger{r,l}(i+1,j,k), and
   *             Bx_stagger{r,l} at (i+1/2,j,k+1/2) is stored at Bx_stagger{r,l}(i,j,k+1).
   * In the case of Az_rhs, we need B{x,y}_stagger{r,l} at (i+1/2,j+1/2,k).
   *    However, Bx_stagger{r,l} is defined at (i+1/2,j-1/2,k), and
   *             By_stagger{r,l} is defined at (i-1/2,j+1/2,k), so
   *             Bx_stagger{r,l} at (i+1/2,j+1/2,k) is stored at Bx_stagger{r,l}(i,j+1,k), and
   *             By_stagger{r,l} at (i+1/2,j+1/2,k) is stored at By_stagger{r,l}(i+1,j,k).
   */
  static const int B1_ijk_offset[4][3] = { {0,0,0} , {0,0,1} , {1,0,0} , {0,1,0} }; // Note that B1_ijk_offset[0] is UNUSED; we choose a 1-offset for convenience.
  static const int B2_ijk_offset[4][3] = { {0,0,0} , {0,1,0} , {0,0,1} , {1,0,0} }; // Note that B2_ijk_offset[0] is UNUSED; we choose a 1-offset for convenience.


Appending to GiRaFFE_standalone_Ccodes/RHSs/A_i_rhs_no_gauge_terms.h


<a id='loop_interior'></a>

## Step 2.b: Loop over the grid interior \[Back to [top](#toc)\]
$$\label{loop_interior}$$

Now, we start a loop over the entire grid interior. We define the memory indices from which we will read the input data, using the offset arrays we just defined. Then, we will read in the values that we need from memory and calculate the flux terms 
$$
\psi^6 \left( v^x B^y - v^y B^x \right)
$$
for all combinations of reconstructions: RR, RL, LR, and LL.

The values of $\psi^6$ were interpolated to the correct locations outside of this function.

We also read in the values of the characteristic speeds at a given point and calculate the difference of the state vectors, $\psi^6 (B^x_{\rm R} - B^x_{\rm L})$ and $\psi^6 (B^y_{\rm R} - B^y_{\rm L})$.

In [5]:
%%writefile -a $Ccodesdir/A_i_rhs_no_gauge_terms.h

#pragma omp parallel for
  for(int k=NGHOSTS;k<Nxx_plus_2NGHOSTS2-NGHOSTS;k++) for(int j=NGHOSTS;j<Nxx_plus_2NGHOSTS1-NGHOSTS;j++) for(int i=NGHOSTS;i<Nxx_plus_2NGHOSTS0-NGHOSTS;i++) {
        const int index=IDX3S(i,j,k);
        // The following lines set the indices appropriately. See justification in exorbitant comments above.
        const int index_v =IDX3S(i+vs_ijk_offset[A_dirn][0],j+vs_ijk_offset[A_dirn][1],k+vs_ijk_offset[A_dirn][2]);
        const int index_B1=IDX3S(i+B1_ijk_offset[A_dirn][0],j+B1_ijk_offset[A_dirn][1],k+B1_ijk_offset[A_dirn][2]);
        const int index_B2=IDX3S(i+B2_ijk_offset[A_dirn][0],j+B2_ijk_offset[A_dirn][1],k+B2_ijk_offset[A_dirn][2]);

        // Stores 1/sqrt(gamma)==exp(6 phi) at (i+1/2,j+1/2,k) for Az, (i+1/2,j,k+1/2) for Ay, and (i,j+1/2,k+1/2) for Az.
        const REAL psi6_interped=psi6_pointer[index];

        const REAL B1lL = B1l[index_B1];
        const REAL B1rL = B1r[index_B1];
        const REAL B2lL = B2l[index_B2];
        const REAL B2rL = B2r[index_B2];

        const REAL A3_rhs_rr = psi6_interped*(v1rr[index_v]*B2rL - v2rr[index_v]*B1rL);
        const REAL A3_rhs_rl = psi6_interped*(v1rl[index_v]*B2rL - v2rl[index_v]*B1lL);
        const REAL A3_rhs_lr = psi6_interped*(v1lr[index_v]*B2lL - v2lr[index_v]*B1rL);
        const REAL A3_rhs_ll = psi6_interped*(v1ll[index_v]*B2lL - v2ll[index_v]*B1lL);


        // All variables for the A_i_rhs computation are now at the appropriate staggered point,
        //   so it's time to compute the HLL flux!

        // Note that with PPM, cmin and cmax are defined between ijk=3 and ijk<cctk_lsh[]-2 for all directions.
        const REAL cmax_1L = cmax_1[index_B2];
        const REAL cmin_1L = cmin_1[index_B2];
        const REAL cmax_2L = cmax_2[index_B1];
        const REAL cmin_2L = cmin_2[index_B1];

        const REAL B1tilder_minus_B1tildel = psi6_interped*( B1rL - B1lL );
        const REAL B2tilder_minus_B2tildel = psi6_interped*( B2rL - B2lL );


Appending to GiRaFFE_standalone_Ccodes/RHSs/A_i_rhs_no_gauge_terms.h


With all the input variables read from memory, we are finally able implement the 2D HLL flux [(Del Zanna et al., 2003)](https://arxiv.org/abs/astro-ph/0210618). 
This is given by 
\begin{align}
-E_z &= \frac{
c_{1{\rm max}} c_{2{\rm max}} A_{z{\rm LL}} + 
c_{1{\rm max}} c_{2{\rm min}} A_{z{\rm LR}} + 
c_{1{\rm min}} c_{2{\rm max}} A_{z{\rm RL}} + 
c_{1{\rm min}} c_{2{\rm min}} A_{z{\rm RR}}
}{(c_{1{\rm max}}+c_{1{\rm min}})(c_{2{\rm max}}+c_{2{\rm min}})} \\
&
- \frac{c_{1{\rm max}} c_{1{\rm min}} (B^y_{\rm R}-B^y_{\rm L})}{c_{1{\rm max}}+c_{1{\rm min}}}
+ \frac{c_{2{\rm max}} c_{2{\rm min}} (B^x_{\rm R}-B^x_{\rm L})}{c_{2{\rm max}}+c_{2{\rm min}}}
\end{align}
(Recall that $\partial_t A_z = -E_z + [{\rm Gauge\ Terms}]$)

In [6]:
%%writefile -a $Ccodesdir/A_i_rhs_no_gauge_terms.h

        /*---------------------------
         * Implement 2D HLL flux
         * [see Del Zanna, Bucciantini & Londrillo A&A 400, 397 (2003), Eq. (44)]
         *
         * Note that cmax/cmin (\alpha^{\pm}  as defined in that paper) is at a slightly DIFFERENT
         * point than that described in the Del Zanna et al paper (e.g., (i+1/2,j,k) instead of
         * (i+1/2,j+1/2,k) for F3).  Yuk Tung Liu discussed this point with M. Shibata,
         * who found that the effect is negligible.
         ---------------------------*/
        A3_rhs[index] = (cmax_1L*cmax_2L*A3_rhs_ll + cmax_1L*cmin_2L*A3_rhs_lr +
                         cmin_1L*cmax_2L*A3_rhs_rl + cmin_1L*cmin_2L*A3_rhs_rr)
          /( (cmax_1L+cmin_1L)*(cmax_2L+cmin_2L) )
          - cmax_1L*cmin_1L*(B2tilder_minus_B2tildel)/(cmax_1L+cmin_1L)
          + cmax_2L*cmin_2L*(B1tilder_minus_B1tildel)/(cmax_2L+cmin_2L);
      }
}


Appending to GiRaFFE_standalone_Ccodes/RHSs/A_i_rhs_no_gauge_terms.h


<a id='code_validation'></a>

# Step 3: Code Validation \[Back to [top](#toc)\]
$$\label{code_validation}$$

To validate the code in this tutorial we check for agreement between the files

1. that were written in this tutorial and
1. those that are generated by the python module


In [7]:
# Define the directory that we wish to validate against:
valdir = "GiRaFFE_NRPy/GiRaFFE_Ccode_library/RHSs/"

import GiRaFFE_NRPy.GiRaFFE_NRPy_staggered_Afield_flux as Af
Af.GiRaFFE_NRPy_Afield_flux(valdir)

import difflib
import sys

print("Printing difference between original C code and this code...")
# Open the files to compare
files = ["A_i_rhs_no_gauge_terms.h"]

for file in files:
    print("Checking file " + file)
    with open(os.path.join(valdir,file)) as file1, open(os.path.join(Ccodesdir,file)) as file2:
        # Read the lines of each file
        file1_lines = file1.readlines()
        file2_lines = file2.readlines()
        num_diffs = 0
        for line in difflib.unified_diff(file1_lines, file2_lines, fromfile=os.path.join(valdir+file), tofile=os.path.join(Ccodesdir+file)):
            sys.stdout.writelines(line)
            num_diffs = num_diffs + 1
        if num_diffs == 0:
            print("No difference. TEST PASSED!")
        else:
            print("ERROR: Disagreement found with .py file. See differences above.")
            sys.exit(1)

Printing difference between original C code and this code...
Checking file A_i_rhs_no_gauge_terms.h
No difference. TEST PASSED!


<a id='latex_pdf_output'></a>

# Step 4: Output this notebook to $\LaTeX$-formatted PDF file \[Back to [top](#toc)\]
$$\label{latex_pdf_output}$$

The following code cell converts this Jupyter notebook into a proper, clickable $\LaTeX$-formatted PDF file. After the cell is successfully run, the generated PDF may be found in the root NRPy+ tutorial directory, with filename
[Tutorial-GiRaFFE_NRPy_staggered-Afield_flux.pdf](Tutorial-GiRaFFE_NRPy_staggered-Afield_flux.pdf) (Note that clicking on this link may not work; you may need to open the PDF file through another means.)

In [8]:
import cmdline_helper as cmd    # NRPy+: Multi-platform Python command-line interface
cmd.output_Jupyter_notebook_to_LaTeXed_PDF("Tutorial-GiRaFFE_NRPy_staggered-Afield_flux",location_of_template_file=os.path.join(".."))

Created Tutorial-GiRaFFE_NRPy_staggered-Afield_flux.tex, and compiled LaTeX
    file to PDF file Tutorial-GiRaFFE_NRPy_staggered-Afield_flux.pdf
